In [1]:
# google drive 연결
# url 파일이 드라이브 안에 있거나 코랩에 올려져 있어야 함
# 최초만 1회만 실행해도 됨
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# selenium & chrome driver 설치
# 최초 1회만 실행하기!
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install chromedriver-autoinstaller

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.3 MB/s eta 0:00:00
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Ign:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics

In [3]:
# Selenium 설치 확인
!python --version
import selenium
print(selenium.__version__)

Python 3.10.12
4.25.0


In [4]:
#Selenium import
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException  # 예외 처리 추가
from selenium.webdriver.chrome.options import Options # user-agent 변경
#pandas import
import pandas as pd
#standard library
import time, os, random, sys, json
# chrome options
import chromedriver_autoinstaller

# Auto Dict
Json 처리할 때 쓰려고 만든 dictionary 자료형 처리 클래스

In [5]:
class AutoDict:
    def __init__(self, existing_data=None):
        self.data = {}
        self.counter = 1  # 기본적으로 1부터 시작하지만 기존 데이터가 있으면 수정
        if existing_data:
            self.load_existing_data(existing_data)

    def load_existing_data(self, existing_data):
        # 기존 JSON 데이터를 로드하고 key에 맞춰 데이터를 채움
        self.data = {int(k): v for k, v in existing_data.items()}
        self.counter = max(self.data.keys()) + 1  # 다음 번호는 가장 큰 key + 1

    def add(self, value):
        # 가장 작은 빈 key를 찾아서 할당
        key = self._get_next_key()
        self.data[key] = value

    def remove(self, key):
        # key가 있는지 확인 후 삭제
        if key in self.data:
            del self.data[key]
            # key를 채우기 위해 재정렬
            self._fill_empty_keys()

    def _get_next_key(self):
        # 빈 key를 찾음
        for i in range(1, len(self.data) + 2):
            if i not in self.data:
                return i

    def _fill_empty_keys(self):
        # key를 순차적으로 채우기 위해 데이터를 재정렬
        values = list(self.data.values())
        self.data = {i + 1: values[i] for i in range(len(values))}

    def __str__(self):
        return str(self.data)

# 데이터 불러오기
여기서 불러올 url 데이터랑 저장할 파일 경로 입력해주면 됨

In [6]:
data = pd.read_csv('/content/drive/Othercomputers/내 노트북/scrapping_darkpattern/data/test.CSV') # Input 파일(URL) 경로
output_file = "/content/drive/Othercomputers/내 노트북/scrapping_darkpattern/result/test.json" # Output 파일 경로

In [7]:
# 기존 데이터가 있는지 확인하고 로드
if os.path.isfile(output_file):
    with open(output_file, 'r', encoding='utf-8') as f:
        try:
            output_data = json.load(f)  # 기존 데이터 읽기
            output_dataset = AutoDict(output_data)  # AutoDict 인스턴스 생성
        except json.JSONDecodeError:  # 파일이 비어 있을 경우 처리
            output_dataset = AutoDict()
else:
    output_dataset = AutoDict()

# function

In [8]:
def search_keyword(keywords, driver, url):
    # keyword가 포함된 block 불러와서 JSON 파일에 저장하는 함수

    # XPath OR 조건 생성
    xpath_conditions = " or ".join([f"contains(text(), '{keyword}')" for keyword in keywords])
    # 최종 XPath
    xpath = f"//*[{xpath_conditions}]"
    # Selenium을 활용해 요소 찾기
    elements = driver.find_elements(By.XPATH, xpath)

    # 데이터 생성
    #3. 데이터 가져오기
    find_div(elements, keywords, url)

In [9]:
# div 레이아웃 가져오기
def find_div(elements, keywords, url):

    for element in elements:
        try:
            # 현재 요소를 포함하는 가장 가까운 div 찾기
            closest_div = element.find_element(By.XPATH, './ancestor::div[1]')
            html_code = closest_div.get_attribute('outerHTML')

            # CSS 스타일 가져오기
            css_styles = driver.execute_script("""
               const parent = arguments[0];
               const allElements = [parent, ...parent.querySelectorAll('*')]; // 모든 자식 요소를 선택
               const styles = {};
               allElements.forEach(element => {
                   const computedStyle = window.getComputedStyle(element);
                   const className = element.className.trim(); // 클래스 이름 가져오기
                   if (className) {
                       const styleObject = {};
                       for (let i = 0; i < computedStyle.length; i++) {
                           const property = computedStyle.item(i);
                           styleObject[property] = computedStyle.getPropertyValue(property);
                      }
                   styles[`.${className.replace(/\s+/g, '.')}`] = styleObject;
                   }
                });
                return styles;
            """, closest_div)

            # 스타일 출력
            style_output = ""
            for class_selector, styles in css_styles.items():  # 딕셔너리의 각 항목
                style_output += f"{class_selector} {{ \n"
                for property, value in styles.items():
                    style_output += f"  {property}: {value};\n"
                style_output += "}\n"

            # 일치하는 키워드 찾기
            matched_keywords = [keyword for keyword in keywords if keyword in html_code]

            # 결과 저장
            output_dataset.add({'HTML': html_code, 'CSS': style_output, 'Matched Keywords': matched_keywords, 'URL': url})

            # 결과 출력
            print("HTML Code:")
            print(html_code)

        except NoSuchElementException:
            print("가장 가까운 div 요소를 찾을 수 없습니다.")

    return

# Scrapper

In [10]:
# Chrome 옵션 설정
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # ensure GUI is off
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')  # set path to chromedriver as per your configuration
chrome_options.add_argument('lang=ko_KR') # 한국어

chrome_options.add_argument("Mozilla/5.0")  # 원하는 User-Agent 문자열로 변경

In [11]:
driver = webdriver.Chrome(options=chrome_options) # chrome driver의 경로가 파일 경로와 같아야 함

In [12]:
# Keyword 설정
price_disrupt = ["실적", "조건", "신한카드", "삼성카드", "KB국민카드", "현대카드", "롯데카드", "우리카드", "하나카드", "BC카드", "NH농협카드"]
hidden_info = [ "세금", "수수료", "청소료", "부가세", "서비스세", "관세", "팁", "관리비", "청소비", "청소료", "정리비", "운영비", "중개료", "중개비"]
emotional_lang = ["마지막", "포기", "정말로", "놓치지", "기회", "지금", "한정", "수량", "곧", "마감", "임박", "가장", "너무", "망설"]
disguised_ad = ["AD", "ad", "Ad"]
trap_question = ["않으"]
false_sale = ["1+1 할인", "1+1할인", "원쁠원 할인", "원플원 할인", "원 쁠 원 할인", "원 플 원 할인"]

keywords_dict = [price_disrupt, hidden_info, emotional_lang, disguised_ad, trap_question, false_sale]

In [13]:
for url in data['URL']:

    #1.url open
    print(url)
    driver.get(url)

    # time.sleep(random.randrange(10, 20)) # 10초 이상 기다려서 로딩되는데 걸리는 시간을 주기
    time.sleep(5)
    for keyword in keywords_dict:
        search_keyword(keyword, driver, url) # element 불러와서 output_dataset에 저장


https://flight.naver.com/flights/international/ICN-BUS-20241113/BUS-ICN-20241120?adult=1&fareType=Y
HTML Code:
<div class="item_defaultSummary__iVehV" itemscope=""><span class="item_type__79id0" itemprop="name">성인/KB국민카드(이용실적 충족시)</span><b class="item_usual__wg002" itemprop="offers">왕복 <i class="item_num__aKbk4">1,802,200</i><span class="item_unit__ZqikD">원~</span></b></div>
HTML Code:
<div class="item_defaultSummary__iVehV" itemscope=""><span class="item_type__79id0" itemprop="name">성인/KB국민카드(이용실적 충족시)</span><b class="item_usual__wg002" itemprop="offers">왕복 <i class="item_num__aKbk4">1,833,500</i><span class="item_unit__ZqikD">원~</span></b></div>
HTML Code:
<div class="item_defaultSummary__iVehV" itemscope=""><span class="item_type__79id0" itemprop="name">성인/KB국민카드(이용실적 충족시)</span><b class="item_usual__wg002" itemprop="offers">왕복 <i class="item_num__aKbk4">1,836,100</i><span class="item_unit__ZqikD">원~</span></b></div>
HTML Code:
<div class="item_defaultSummary__iVehV" itemscope=""><s

In [14]:
# JSON으로 바꿔서 저장하기
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(output_dataset.data, f, ensure_ascii=False, indent=4)

In [15]:
driver.quit()